In [11]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import log_loss, f1_score, precision_score, recall_score
from lightgbm import LGBMClassifier
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import ColumnTransformer

pd.options.mode.chained_assignment = None

# 1. Carregando Dados

In [2]:
df_train = pd.read_csv('../dataset/df_train_feature_churn.csv')
df_train

,created_at,customer_id,payment_method,promo_amount,shipment_fee,total_amount,churn,time_purchase_delivery,delivery_percent_total_amount,is_shipment_fee,is_promo_code,quant_prod_purchased
0,2018-07-29,5868,Debit Card,1415,10000,199832,1,5,5.004204,1,1,1
1,2018-07-30,4774,Credit Card,0,10000,155526,0,4,6.429793,1,0,1
2,2018-09-15,4774,OVO,0,10000,550696,0,3,1.815884,1,0,4
3,2018-11-01,4774,Credit Card,0,0,271012,0,4,0.000000,0,0,1
4,2018-12-18,4774,Credit Card,0,0,198753,0,5,0.000000,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
816035,2020-11-12,25121,Debit Card,0,10000,287688,1,6,3.475988,1,0,1
816036,2021-12-09,34829,OVO,5010,10000,359203,1,3,2.783941,1,1,1
816037,2021-12-11,99645,Credit Card,2181,5000,1158159,1,1,0.431720,1,1,4
816038,2019-12-18,87319,Credit Card,4503,0,184143,1,5,0.000000,0,1,1


In [3]:
X = df_train.drop(columns=['created_at', 'customer_id', 'churn'])
y = df_train[['churn']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, shuffle=True, random_state=2023)

In [4]:
X_train

,payment_method,promo_amount,shipment_fee,total_amount,time_purchase_delivery,delivery_percent_total_amount,is_shipment_fee,is_promo_code,quant_prod_purchased
162775,LinkAja,2616,10000,295440,5,3.384782,1,1,1
744796,Credit Card,5974,0,349955,2,0.000000,0,1,1
589954,Gopay,0,0,671814,5,0.000000,0,0,6
814560,Credit Card,0,10000,136594,4,7.320966,1,0,1
47304,OVO,9268,10000,287768,5,3.475022,1,1,1
...,...,...,...,...,...,...,...,...,...
533984,Credit Card,0,0,524545,3,0.000000,0,0,2
782331,Credit Card,0,0,125168,5,0.000000,0,0,1
333589,LinkAja,0,10000,472359,5,2.117034,1,0,1
608748,Gopay,0,50000,475705,5,10.510716,1,0,1


In [12]:
from sklearn.base import TransformerMixin #gives fit_transform method for free
class MyLabelEncoder(TransformerMixin):
    def __init__(self, *args, **kwargs):
        self.encoder = LabelEncoder(*args, **kwargs)
    
    def fit(self, x, y=0):
        self.encoder.fit(x)
        return self
    
    def transform(self, x, y=0):
        return self.encoder.transform(x)



d:\pasta_virtualenv\churn_prediction_env\lib\site-packages\sklearn\preprocessing\_label.py:99: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
d:\pasta_virtualenv\churn_prediction_env\lib\site-packages\sklearn\preprocessing\_label.py:134: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
d:\pasta_virtualenv\churn_prediction_env\lib\site-packages\sklearn\pipeline.py:405: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params_last_step)


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('categorical',
                                                  Pipeline(steps=[('label_encoder',
                                                                   <__main__.MyLabelEncoder object at 0x000001C72FF839A0>),
                                                                  ('reshape',
                                                                   FunctionTransformer(func=<function <lambda> at 0x000001C72D856050>))]),
                                                  ['payment_method'])])),
                ('mdl', RandomForestClassifier())])

In [5]:
categorical_features = ['payment_method']
categorical_transformer = Pipeline(steps=[('label_encoder', LabelEncoder())])

preprocessor = ColumnTransformer(transformers=[('cat', categorical_transformer, categorical_features)])
pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('model', RandomForestClassifier())])

pipeline.fit(X_train, y_train)

TypeError: LabelEncoder.fit_transform() takes 2 positional arguments but 3 were given

In [24]:
def execute_cross_validation_stratified(mdl, X, y):
    skf = StratifiedKFold(n_splits=5, random_state=2023, shuffle=True)
    
    list_f1 = []
    list_precision = []
    list_recall = []
    list_logloss = []
    
    #cat_features = ['payment_method']
    #cat_transformer = Pipeline(steps=[('label_encoder', OneHotEncoder(sparse_output=False))])
    #preprocessor = ColumnTransformer(transformers=[('categorical', cat_transformer, cat_features)])
    
    #categorical_features = ['payment_method']
    #categorical_transformer = Pipeline(steps=[('label_encoder', MyLabelEncoder()), 
    #                                          ('reshape', FunctionTransformer(lambda x: x.reshape(-1, 1)))])
    
    #preprocessor = ColumnTransformer(transformers=[('categorical', cat_transformer, cat_features)])
    #mdl_pipe = Pipeline(steps=[('preprocessor', preprocessor), ('mdl', mdl)])
    
    for train_index, test_index in skf.split(X, y):
        X_train_cross, y_train_cross = X.iloc[train_index], y.iloc[train_index]
        X_test_cross, y_test_cross = X.iloc[test_index], y.iloc[test_index]
        
        l_encoder = LabelEncoder()
        #print(X_train_cross)
    
        X_train_cross['payment_method_label'] = l_encoder.fit_transform(X_train_cross['payment_method'])
        X_test_cross['payment_method_label'] = l_encoder.transform(X_test_cross['payment_method'])
    
        X_train_cross.drop(columns=['payment_method'], inplace=True)
        X_test_cross.drop(columns=['payment_method'], inplace=True)
        
        mdl.fit(X_train_cross, y_train_cross.values.ravel())
        
        pred = mdl.predict(X_test_cross)
        
        list_f1.append(f1_score(y_test_cross, pred))
        list_precision.append(precision_score(y_test_cross, pred))
        list_recall.append(recall_score(y_test_cross, pred))
        list_logloss.append(log_loss(y_test_cross, pred))
        
    print(f"F1 Score:{np.mean(list_f1)}")
    print(f"Precison Score:{np.mean(list_precision)}")
    print(f"Recall Score:{np.mean(list_recall)}")
    print(f"Log Loss Score:{np.mean(list_logloss)}")

In [25]:
# Dummy
mdl = DummyClassifier()
execute_cross_validation_stratified(mdl, X_train, y_train)

d:\pasta_virtualenv\churn_prediction_env\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\pasta_virtualenv\churn_prediction_env\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\pasta_virtualenv\churn_prediction_env\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\pasta_virtualenv\churn_prediction_env\lib\site-packages\sklearn\met

F1 Score:0.0
Precison Score:0.0
Recall Score:0.0
Log Loss Score:8.998609131504232


d:\pasta_virtualenv\churn_prediction_env\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [26]:
# Regression Logistic
mdl = LogisticRegression()
execute_cross_validation_stratified(mdl, X_train, y_train)

d:\pasta_virtualenv\churn_prediction_env\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\pasta_virtualenv\churn_prediction_env\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\pasta_virtualenv\churn_prediction_env\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\pasta_virtualenv\churn_prediction_env\lib\site-packages\sklearn\met

F1 Score:0.0
Precison Score:0.0
Recall Score:0.0
Log Loss Score:8.998609131504232


d:\pasta_virtualenv\churn_prediction_env\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# Random Forest
mdl = RandomForestClassifier()
execute_cross_validation_stratified(mdl, X_train, y_train)

In [ ]:
#LGBM
mdl = LGBMClassifier()
execute_cross_validation_stratified(mdl, X_train, y_train)

In [ ]:
#XGBoost
mdl = XGBClassifier()
execute_cross_validation_stratified(mdl, X_train, y_train)

In [ ]:
#CatBoost
mdl = CatBoostClassifier(verbose=False)
execute_cross_validation_stratified(mdl, X_train, y_train)

Vou levar em consideração a métrica F1-Score o que faz o algoritmo Random Forest ser considerado o melhor com 19%